In [1]:
import pandas as pd 
df = pd.read_csv('./df.csv')
df_test = pd.read_csv('./df_test.csv')

In [ ]:
def transform(df):
    import numpy as np
    features = df.columns.tolist()
    sorted_new =  np.sort(df.values, axis=1)
    # df['mean_features'] = 0.1 * df[features].mean(axis=1)
    # df['std_features'] = df[features].std(axis=1)
    # df['max_features'] = df[features].max(axis=1)
    # df['min_features'] = df[features].min(axis=1)
    # df['median_features'] = 0.1 * df[features].median(axis=1)
    # df['sum_features'] = 0.1 * df[features].sum(axis=1)
    num_cols = features
    df['sum'] = df[num_cols].sum(axis=1)
    df['std']  = df[num_cols].std(axis=1)
    df['mean'] = df[num_cols].mean(axis=1)
    df['max']  = df[num_cols].max(axis=1)
    df['min']  = df[num_cols].min(axis=1)
    df['mode'] = df[num_cols].mode(axis=1)[0]
    # df['median'] = df[num_cols].median(axis=1)
    df['q_25th'] = df[num_cols].quantile(0.25, axis=1)
    df['q_75th'] = df[num_cols].quantile(0.75, axis=1)
    df['skew'] = df[num_cols].skew(axis=1)
    df['kurt'] = df[num_cols].kurt(axis=1)
    df['sum_72_76'] = df['sum'].isin(np.arange(72, 76))
    # for i in range(10,100,10):
    #     df[f'{i}th'] = df[num_cols].quantile(i/100, axis=1)
    df['harmonic'] = len(num_cols) / df[num_cols].apply(lambda x: (1/x).mean(), axis=1)
    df['geometric'] = df[num_cols].apply(lambda x: x.prod()**(1/len(x)), axis=1)
    df['zscore'] = df[num_cols].apply(lambda x: (x - x.mean()) / x.std(), axis=1).mean(axis=1)
    df['cv'] = df['std'] / df['mean']
    df['Skewness_75'] = (df[num_cols].quantile(0.75, axis=1) - df[num_cols].mean(axis=1)) / df[num_cols].std(axis=1)
    df['Skewness_25'] = (df[num_cols].quantile(0.25, axis=1) - df[num_cols].mean(axis=1)) / df[num_cols].std(axis=1)
    df['2ndMoment'] = df[num_cols].apply(lambda x: (x**2).mean(), axis=1)
    df['3rdMoment'] = df[num_cols].apply(lambda x: (x**3).mean(), axis=1)
    # df['entropy'] = df[num_cols].apply(lambda x: -1*(x*np.log(x)).sum(), axis=1)
    # df1 = pd.concat([df, pd.DataFrame(sorted_new, index=df.index)], axis=1)
    # df1 = df1.drop(features, axis=1)
    # df1.columns = df1.columns.astype('str')
    return df

df_train = transform(df.drop(['id','FloodProbability'],axis=1))

df_test = transform(df_test.drop(['id'],axis=1))
df_train

In [ ]:
df = df_train
df_test = df_test

In [ ]:
# 删除重复
df=df.drop_duplicates()

In [ ]:
df_test # 测试集数量  745305

# 特征处理

In [ ]:
df_train = df.sample(frac=0.6)
df_train

In [6]:
train_x,train_y,test_x,n_jobs = df.drop('FloodProbability',axis=1),df['FloodProbability'],df_test,8

In [7]:
#1st 方案

#控制特征生成器
all_features = list(train_data.columns)
soft_ordinal = [f for f in all_features if (train_data[f].nunique() <= 100) and (f not in ordinal_columns)]
numerical_features = [f for f in all_features if f not in categorical_columns]
candidate_features = get_candidate_features(
    numerical_features=numerical_features,
    categorical_features=categorical_columns,
    ordinal_features=ordinal_columns + soft_ordinal,
    order=1,  # 2 is likely impossible to use w/o time estimate.
)

# Restrict Search Space of Candidate Features
candidate_features = [
    f
    for f in candidate_features
    if f.name
    in [
        # "abs" -> dataset specific, not useful in most cases
        # "log" -> can be done by scalers, no need for GBDTs
        # "sqrt", -> see above (s.a.)
        # "square" , -> s.a.
        # "sigmoid" , -> s.a.
        "freq", -> # do not like it but giving it a shot. 
        "round",
        "residual",
        # "max", -> IMO, trivial to model for first-order features 
        # "min", -> s.a.
        # "+", -> s.a.
        # "-", -> s.a.
        "/",
        "*",
        # "GroupByThenMin",  -> &nbsp;The essential benefit of GroupBy is captured with any of these, so I filtered this to reduce the search space.
        # "GroupByThenMax", -> s.a.
        # "GroupByThenMean", -> s.a.
        "GroupByThenMedian",
        "GroupByThenStd",
        # "GroupByThenRank", -> s.a.
        "GroupByThenFreq",
        "GroupByThenNUnique",
        "Combine",
        # New Generators 
        #   - Hacked into OpenFE by adding `new_data = int(d < d.quantile(X).max())` to the generator options. 
         "<p0.2",  # X = 0.2
    "<p0.4",   
    "<p0.6",
    "<p0.8",
    ]
]

SyntaxError: invalid syntax (987350402.py, line 25)

这段描述概括了一个自动机器学习(AutoML)工作流程，该流程包括以下几个关键步骤，旨在优化模型训练数据、特征工程、模型选择等，以提高在Kaggle竞赛中的表现。以下是对其内容的分析及未来可能的改进方向：

### 1. 自动化的两样本测试优化训练数据

- **目标**：评估外部数据源（"原始数据"）与竞赛提供的训练和测试数据集之间是否存在分布差异（分布偏移）。分布偏移的存在可能意味着合并原始数据到训练集中可能不会提升模型泛化能力，甚至可能有害。
  
- **方法**：
  - **AutoGluon分类**：利用AutoGluon框架尝试建立一个分类模型，预测样本是来自哪个数据集（例如，区分原数据与竞赛数据集），以此作为检测数据集间差异的一种方式。
  - **曼-惠特尼U检验**（Mann-Whitney U Test）：这是一种非参数统计检验，用于判断两个独立样本是否来自具有相同分布的总体，帮助量化数据集之间的差异是否显著。

- **测试场景**：
  - (Kaggle+Original) Train vs (Kaggle) Test：评估加入原数据后的训练集与竞赛测试集的差异。
  - (Kaggle) Train vs (Original) Train：直接比较竞赛训练集与原训练集。
  - (Kaggle) Train vs (Kaggle) Test：基础对比，验证竞赛内部数据的一致性。
  - (Original) Train vs (Kaggle) Test：评估原数据与竞赛测试集的差异。

- **结果处理**：基于测试结果，决定是否将原数据纳入训练。目前是基于观察手动决策，但表达了自动化此决策过程的需求。

### 未来改进方向：

- **自动决策机制**：开发一套算法或规则，根据两样本测试的结果自动判断是否合并数据。这可能涉及到设定阈值，如当统计检验的p值低于某个特定阈值时才认为数据可合并，或者基于模型性能预测的变化来决定。

- **性能预估**：在决定是否整合数据之前，可以先通过交叉验证或类似技术预测加入新数据后模型性能的潜在变化。这可能需要构建一个初步模型，并在不同数据组合上评估其性能，选择最优方案。

- **更广泛的测试套件**：除了现有的测试，还可以探索其他统计检验或机器学习方法来更全面地评估数据质量、相关性和分布一致性，比如KS检验、t检验、特征重要性分析等。

- **特征工程自动化**：既然提到了OpenFE和AutoGluon，可以进一步优化自动化特征工程过程，确保它能有效生成对模型性能有显著提升的新特征，并且能够高效处理大数据量。

- **模型选择优化**：定制化的AutoGluon或集成学习策略可以被进一步优化，比如引入更多的超参数调优策略、模型投票机制或更智能的模型组合方法，以找到针对特定任务的最佳模型配置。

总之，未来的重点在于增强自动化决策能力，通过更精细化的测试和预测模型来智能指导训练数据的选择、特征工程和模型选择过程，以实现更高效、性能更优的AutoML解决方案。

In [8]:
len(df_train)

11180

In [25]:
pipline(df.sample(frac=0.6),df_test,label='FloodProbability',loss='r2',time=60*60*6,id_name='flud_0620_v8',cols=['id','FloodProbability'])

No path specified. Models will be saved in: "AutogluonModels\ag-20240622_090648"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.9.19
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22000
CPU Count:          8
Memory Avail:       17.72 GB / 31.95 GB (55.5%)
Disk Space Avail:   157.25 GB / 421.00 GB (37.4%)
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=10, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is used to detect stacked overfitting.
	Running DyStack for up to 900s of the 3600s of remaining time (25%).
		Contex

++++++++++++++++++++++++++++++++++++++++++++++
*** Summary of fit() ***
Estimated performance of each model:
                 model  score_val eval_metric  pred_time_val     fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0  WeightedEnsemble_L3   0.847105          r2      58.966583  2574.787760                0.039892           3.351737            3       True          4
1       XGBoost_BAG_L2   0.847054          r2      58.926690  2571.436023               18.791847         842.254426            2       True          3
2       XGBoost_BAG_L1   0.835994          r2      36.397569  1592.154549               36.397569        1592.154549            1       True          1
3   XGBoost_r33_BAG_L1   0.496903          r2       3.737274   137.027048                3.737274         137.027048            1       True          2
Number of models trained: 4
Types of models trained:
{'WeightedEnsembleModel', 'StackerEnsembleModel_XGBoost'}
Bagging used: True  

D:\programesx64\Anaconda3\envs\mykaggle\lib\site-packages\autogluon\core\utils\plots.py:169: UserWarning: AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"
  warnings.warn('AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"')
These features in provided data are not utilized by the predictor and will be ignored: ['flud_0620_v8']


                 model  score_test  score_val eval_metric  pred_time_test  \
0       XGBoost_BAG_L2    0.882485   0.847054          r2      902.341875   
1  WeightedEnsemble_L3    0.882256   0.847105          r2      902.355837   
2       XGBoost_BAG_L1    0.869540   0.835994          r2      615.551393   
3   XGBoost_r33_BAG_L1    0.534827   0.496903          r2       70.946882   

   pred_time_val     fit_time  pred_time_test_marginal  \
0      58.926690  2571.436023               215.843600   
1      58.966583  2574.787760                 0.013962   
2      36.397569  1592.154549               615.551393   
3       3.737274   137.027048                70.946882   

   pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  \
0               18.791847         842.254426            2       True   
1                0.039892           3.351737            3       True   
2               36.397569        1592.154549            1       True   
3                3.737274         1

0         0.575260
1         0.460364
2         0.464986
3         0.471245
4         0.461084
            ...   
745300    0.476568
745301    0.454052
745302    0.623289
745303    0.548393
745304    0.514689
Name: FloodProbability, Length: 745305, dtype: float32

In [26]:
!kaggle competitions submit -c playground-series-s4e5 -f submission_flud_0620_v8.csv -m "0621_v8_flud"

Successfully submitted to Regression with a Flood Prediction Dataset


  0%|          | 0.00/13.9M [00:00<?, ?B/s]
  0%|          | 8.00k/13.9M [00:00<21:20, 11.4kB/s]
 25%|██▍       | 3.44M/13.9M [00:00<00:01, 5.95MB/s]
 39%|███▉      | 5.45M/13.9M [00:01<00:01, 6.74MB/s]
 53%|█████▎    | 7.30M/13.9M [00:01<00:00, 7.45MB/s]
 75%|███████▍  | 10.4M/13.9M [00:01<00:00, 7.99MB/s]
 90%|████████▉ | 12.4M/13.9M [00:02<00:00, 6.97MB/s]
100%|██████████| 13.9M/13.9M [00:04<00:00, 3.33MB/s]


In [23]:
def feature_engineering(df,df_test,target,n_jobs=4):
    from openfe import OpenFE, transform
    train_x,test_x,train_y,n_jobs = df.drop(target,axis=1),df_test,df[target],n_jobs
    ofe = OpenFE()
    features = ofe.fit(data=train_x, label=train_y, n_jobs=n_jobs)  # generate new features
    train_x, test_x = transform(train_x, test_x, features, n_jobs=n_jobs) # transform the train and test data according to generated features.
    return train_x, test_x

def ag_tuning(df_train,df_test,id_name='0620',label='Survived',loss='log_loss',presets='best_quality',dynamic_stacking=True,folds=10,time=60*3):
    from autogluon.tabular import TabularPredictor
    from autogluon.features.generators import AutoMLPipelineFeatureGenerator
    auto_ml_pipeline_feature_generator = AutoMLPipelineFeatureGenerator()
    from autogluon.tabular.configs.hyperparameter_configs import get_hyperparameter_config
    
    custom_hyperparameters = get_hyperparameter_config('default')
    
    def get_oof_ts(predictor,_train_x,_test_x,ids,label):
        predictions_oof = predictor.predict_oof()
        prdict_y = predictor.predict(_test_x)
        pd.DataFrame({'oofs': predictions_oof, 'predict_y': prdict_y}).to_csv(f'model_hc_systhis_{ids}.csv',index=False)
        _train_x[ids],_test_x[ids]  = predictions_oof,prdict_y
        _train_x.to_csv(f'model_hc_tr_{ids}.csv',index=False)
        _test_x.to_csv(f'model_hc_ts_{ids}.csv',index=False)
        return predictions_oof,prdict_y
    
    predictor = TabularPredictor(
        label=label,
        # problem_type=problem,#'binary',
        eval_metric=loss #'acc'
    ).fit(
        df_train, #df, #df_train,
        presets='best_quality',
    #     unlabeled_data = df_test2,   #df_test2.drop(columns='Survived'), #df_train[800:].drop(columns='Survived'),
        # auto_stack=True,
        feature_generator=auto_ml_pipeline_feature_generator,
        ag_args_ensemble = {'fold_fitting_strategy':'sequential_local'},
        dynamic_stacking=dynamic_stacking,
    #     num_stack_levels=1,
        num_bag_folds=folds,
        # time_limit=time,
        included_model_types = ['XGB','NN_TORCH','CATBOOST','LIGHTGBM','FAST_AI'],
    #     excluded_model_types = ['NN_TORCH'],
        # use_bag_holdout=True,
        # hyperparameters=custom_hyperparameters,
        # hyperparameter_tune_kwargs='auto',#hyperparameter_tune_kwargs,
        keep_only_best = True,
    )
    oofs,predictions = get_oof_ts(predictor,df_train,df_test,id_name,label)
    return predictions,oofs,predictor
# show_info(predictor,train_x)

def show_info(predictor,df_train):
    print('++++++++++++++++++++++++++++++++++++++++++++++')
    print(predictor.fit_summary())
    print('++++++++++++++++++++++++++++++++++++++++++++++')
    print(predictor.leaderboard(df_train,silent=True))
    print('++++++++++++++++++++++++++++++++++++++++++++++')
    print(predictor.feature_importance(df_train,silent=True))
    print('++++++++++++++++++++++++++++++++++++++++++++++')

from sklearn.metrics import mean_squared_error,roc_auc_score
def hill_climbers(files,train_path,test_path,label='Survived',func=roc_auc_score):
    from hillclimbers import climb_hill, partial
    
    train,test = pd.read_csv(train_path),pd.read_csv(test_path)
    oofs,pred = pd.DataFrame(),pd.DataFrame()
    for i in range(len(files)):
        tmp = pd.read_csv(files[i])
        oofs[f'm{i}'],pred[f'm{i}'] = tmp['oofs'].dropna(),tmp['predict_y'].dropna()

    test_preds, oof_preds = climb_hill(
        train=train, 
        oof_pred_df=oofs, 
        test_pred_df=pred,
        target=label,
        objective="maximize",
        eval_metric=partial(func),
        negative_weights=True, 
        precision=0.001,
        plot_hill=True, 
        plot_hist=True,
        return_oof_preds=True
    )
    return test_preds, oof_preds
# a,b = hill_climbers(files,train_path,test_path=test_path,label='Survived')

def titanic_acc(predictions):
    from sklearn.metrics import accuracy_score
    acc= pd.read_csv('./acc.csv')
    acc_ans = acc.Survived
    print(accuracy_score(acc_ans,predictions)) 

def submission(predictions,test_x,cols,label,id_name):
    test_x[label] = predictions
    test_x[cols].to_csv(f'submission_{id_name}.csv',index=False)
    
    
def pipline(df1,df1_test,label='Survived',time=60*3,id_name='0620_v3',loss='log_loss',cols=['1','2']):
    # train_x, test_x = feature_engineering(df1,df1_test,label)
    # train_x[label] = df1[label]
    # train_x.to_csv('tr.csv',index=False)
    # test_x.to_csv('ts.csv',index=False)
    train_x,test_x = df1,df1_test
    predictions,oofs,predictor = ag_tuning(train_x,test_x,id_name=id_name,time=time,loss=loss,label=label)
    show_info(predictor,train_x)
    # titanic_acc(predictions)
    submission(predictions,test_x,cols,label,id_name)
    return predictions


In [ ]:
!kaggle competitions submit -c playground-series-s4e5 -f submission_flud_0620_v2.csv -m "0621_v1_flud"

In [ ]:
submission_flud_0620_v1.csv

In [5]:
df_solve = pd.concat([df.drop('FloodProbability',axis=1).copy(),df_test],axis=0)
df_solve = df_solve.reset_index(drop=True)
df_solve

,id,MonsoonIntensity,TopographyDrainage,RiverManagement,Deforestation,Urbanization,ClimateChange,DamsQuality,Siltation,AgriculturalPractices,...,IneffectiveDisasterPreparedness,DrainageSystems,CoastalVulnerability,Landslides,Watersheds,DeterioratingInfrastructure,PopulationScore,WetlandLoss,InadequatePlanning,PoliticalFactors
0,0,5,8,5,8,6,4,4,3,3,...,2,5,3,3,5,4,7,5,7,3
1,1,6,7,4,4,8,8,3,5,4,...,9,7,2,0,3,5,3,3,4,3
2,2,6,5,6,7,3,7,1,5,4,...,6,7,3,7,5,6,8,2,3,3
3,3,3,4,6,5,4,8,4,7,6,...,5,2,4,7,4,4,6,5,7,5
4,4,5,3,2,6,4,4,3,3,3,...,5,2,2,6,6,4,1,2,3,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1863257,1863257,5,4,8,3,5,4,4,5,5,...,5,6,1,3,5,6,4,4,6,6
1863258,1863258,4,4,2,12,4,3,4,3,5,...,3,7,4,4,3,5,5,3,5,4
1863259,1863259,5,7,9,5,5,6,7,5,5,...,6,11,3,11,4,5,9,5,5,4
1863260,1863260,4,7,6,3,5,2,3,8,6,...,6,6,8,6,2,3,8,7,5,5


In [6]:
# 添加缺失特征
from feature_engine.imputation import AddMissingIndicator

ami = AddMissingIndicator()
ami.fit(df_solve)
df_solve_mi = ami.transform(df_solve)
df_solve_mi

,id,MonsoonIntensity,TopographyDrainage,RiverManagement,Deforestation,Urbanization,ClimateChange,DamsQuality,Siltation,AgriculturalPractices,...,IneffectiveDisasterPreparedness,DrainageSystems,CoastalVulnerability,Landslides,Watersheds,DeterioratingInfrastructure,PopulationScore,WetlandLoss,InadequatePlanning,PoliticalFactors
0,0,5,8,5,8,6,4,4,3,3,...,2,5,3,3,5,4,7,5,7,3
1,1,6,7,4,4,8,8,3,5,4,...,9,7,2,0,3,5,3,3,4,3
2,2,6,5,6,7,3,7,1,5,4,...,6,7,3,7,5,6,8,2,3,3
3,3,3,4,6,5,4,8,4,7,6,...,5,2,4,7,4,4,6,5,7,5
4,4,5,3,2,6,4,4,3,3,3,...,5,2,2,6,6,4,1,2,3,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1863257,1863257,5,4,8,3,5,4,4,5,5,...,5,6,1,3,5,6,4,4,6,6
1863258,1863258,4,4,2,12,4,3,4,3,5,...,3,7,4,4,3,5,5,3,5,4
1863259,1863259,5,7,9,5,5,6,7,5,5,...,6,11,3,11,4,5,9,5,5,4
1863260,1863260,4,7,6,3,5,2,3,8,6,...,6,6,8,6,2,3,8,7,5,5


In [7]:
#频率编码


df_solve_mi_cf = df_solve_mi.copy()
df_solve_mi_cf

,id,MonsoonIntensity,TopographyDrainage,RiverManagement,Deforestation,Urbanization,ClimateChange,DamsQuality,Siltation,AgriculturalPractices,...,IneffectiveDisasterPreparedness,DrainageSystems,CoastalVulnerability,Landslides,Watersheds,DeterioratingInfrastructure,PopulationScore,WetlandLoss,InadequatePlanning,PoliticalFactors
0,0,5,8,5,8,6,4,4,3,3,...,2,5,3,3,5,4,7,5,7,3
1,1,6,7,4,4,8,8,3,5,4,...,9,7,2,0,3,5,3,3,4,3
2,2,6,5,6,7,3,7,1,5,4,...,6,7,3,7,5,6,8,2,3,3
3,3,3,4,6,5,4,8,4,7,6,...,5,2,4,7,4,4,6,5,7,5
4,4,5,3,2,6,4,4,3,3,3,...,5,2,2,6,6,4,1,2,3,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1863257,1863257,5,4,8,3,5,4,4,5,5,...,5,6,1,3,5,6,4,4,6,6
1863258,1863258,4,4,2,12,4,3,4,3,5,...,3,7,4,4,3,5,5,3,5,4
1863259,1863259,5,7,9,5,5,6,7,5,5,...,6,11,3,11,4,5,9,5,5,4
1863260,1863260,4,7,6,3,5,2,3,8,6,...,6,6,8,6,2,3,8,7,5,5


In [8]:
#分类空白插值
from feature_engine.imputation import MeanMedianImputer
mmi = MeanMedianImputer(imputation_method='median')
mmi.fit(df_solve_mi_cf)
df_solve_mi_cf_mmi = mmi.transform(df_solve_mi_cf)

df_solve_mi_cf_mmi

,id,MonsoonIntensity,TopographyDrainage,RiverManagement,Deforestation,Urbanization,ClimateChange,DamsQuality,Siltation,AgriculturalPractices,...,IneffectiveDisasterPreparedness,DrainageSystems,CoastalVulnerability,Landslides,Watersheds,DeterioratingInfrastructure,PopulationScore,WetlandLoss,InadequatePlanning,PoliticalFactors
0,0,5,8,5,8,6,4,4,3,3,...,2,5,3,3,5,4,7,5,7,3
1,1,6,7,4,4,8,8,3,5,4,...,9,7,2,0,3,5,3,3,4,3
2,2,6,5,6,7,3,7,1,5,4,...,6,7,3,7,5,6,8,2,3,3
3,3,3,4,6,5,4,8,4,7,6,...,5,2,4,7,4,4,6,5,7,5
4,4,5,3,2,6,4,4,3,3,3,...,5,2,2,6,6,4,1,2,3,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1863257,1863257,5,4,8,3,5,4,4,5,5,...,5,6,1,3,5,6,4,4,6,6
1863258,1863258,4,4,2,12,4,3,4,3,5,...,3,7,4,4,3,5,5,3,5,4
1863259,1863259,5,7,9,5,5,6,7,5,5,...,6,11,3,11,4,5,9,5,5,4
1863260,1863260,4,7,6,3,5,2,3,8,6,...,6,6,8,6,2,3,8,7,5,5


In [9]:
#特征扩展
from feature_engine.creation import CyclicalFeatures
cf = CyclicalFeatures()
cf.fit(df_solve_mi_cf_mmi)
df_solve_mi_cf_mmi_v1 = cf.transform(df_solve_mi_cf_mmi)
df_solve_mi_cf_mmi_v1

,id,MonsoonIntensity,TopographyDrainage,RiverManagement,Deforestation,Urbanization,ClimateChange,DamsQuality,Siltation,AgriculturalPractices,...,DeterioratingInfrastructure_sin,DeterioratingInfrastructure_cos,PopulationScore_sin,PopulationScore_cos,WetlandLoss_sin,WetlandLoss_cos,InadequatePlanning_sin,InadequatePlanning_cos,PoliticalFactors_sin,PoliticalFactors_cos
0,0,5,8,5,8,6,4,4,3,3,...,0.995734,0.092268,0.735724,-0.677282,0.989821,0.142315,0.382683,-9.238795e-01,0.923880,3.826834e-01
1,1,6,7,4,4,8,8,3,5,4,...,0.961826,-0.273663,0.837166,0.546948,0.755750,0.654861,1.000000,6.123234e-17,0.923880,3.826834e-01
2,2,6,5,6,7,3,7,1,5,4,...,0.798017,-0.602635,0.475947,-0.879474,0.540641,0.841254,0.923880,3.826834e-01,0.923880,3.826834e-01
3,3,3,4,6,5,4,8,4,7,6,...,0.995734,0.092268,0.915773,-0.401695,0.989821,0.142315,0.382683,-9.238795e-01,0.923880,-3.826834e-01
4,4,5,3,2,6,4,4,3,3,3,...,0.995734,0.092268,0.324699,0.945817,0.540641,0.841254,0.923880,3.826834e-01,0.923880,-3.826834e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1863257,1863257,5,4,8,3,5,4,4,5,5,...,0.798017,-0.602635,0.969400,0.245485,0.909632,0.415415,0.707107,-7.071068e-01,0.707107,-7.071068e-01
1863258,1863258,4,4,2,12,4,3,4,3,5,...,0.961826,-0.273663,0.996584,-0.082579,0.755750,0.654861,0.923880,-3.826834e-01,1.000000,6.123234e-17
1863259,1863259,5,7,9,5,5,6,7,5,5,...,0.961826,-0.273663,0.164595,-0.986361,0.989821,0.142315,0.923880,-3.826834e-01,1.000000,6.123234e-17
1863260,1863260,4,7,6,3,5,2,3,8,6,...,0.895163,0.445738,0.475947,-0.879474,0.909632,-0.415415,0.923880,-3.826834e-01,0.923880,-3.826834e-01


In [10]:
from feature_engine.selection import SmartCorrelatedSelection

scs = SmartCorrelatedSelection(threshold=0.7)
df_solve_mi_cf_mmi_select = scs.fit_transform(df_solve_mi_cf_mmi_v1)
df_solve_mi_cf_mmi_select

,id,MonsoonIntensity,TopographyDrainage,RiverManagement,Deforestation,PoliticalFactors,id_cos,MonsoonIntensity_sin,TopographyDrainage_sin,RiverManagement_sin,...,Watersheds_cos,DeterioratingInfrastructure_sin,DeterioratingInfrastructure_cos,PopulationScore_sin,PopulationScore_cos,WetlandLoss_sin,WetlandLoss_cos,InadequatePlanning_sin,InadequatePlanning_cos,PoliticalFactors_sin
0,0,5,8,5,8,3,1.0,0.923880,0.342020,9.238795e-01,...,-3.826834e-01,0.995734,0.092268,0.735724,-0.677282,0.989821,0.142315,0.382683,-9.238795e-01,0.923880
1,1,6,7,4,4,3,1.0,0.707107,0.642788,1.000000e+00,...,3.826834e-01,0.961826,-0.273663,0.837166,0.546948,0.755750,0.654861,1.000000,6.123234e-17,0.923880
2,2,6,5,6,7,3,1.0,0.707107,0.984808,7.071068e-01,...,-3.826834e-01,0.798017,-0.602635,0.475947,-0.879474,0.540641,0.841254,0.923880,3.826834e-01,0.923880
3,3,3,4,6,5,5,1.0,0.923880,0.984808,7.071068e-01,...,6.123234e-17,0.995734,0.092268,0.915773,-0.401695,0.989821,0.142315,0.382683,-9.238795e-01,0.923880
4,4,5,3,2,6,5,1.0,0.923880,0.866025,7.071068e-01,...,-7.071068e-01,0.995734,0.092268,0.324699,0.945817,0.540641,0.841254,0.923880,3.826834e-01,0.923880
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1863257,1863257,5,4,8,3,6,1.0,0.923880,0.984808,1.224647e-16,...,-3.826834e-01,0.798017,-0.602635,0.969400,0.245485,0.909632,0.415415,0.707107,-7.071068e-01,0.707107
1863258,1863258,4,4,2,12,4,1.0,1.000000,0.984808,7.071068e-01,...,3.826834e-01,0.961826,-0.273663,0.996584,-0.082579,0.755750,0.654861,0.923880,-3.826834e-01,1.000000
1863259,1863259,5,7,9,5,4,1.0,0.923880,0.642788,-3.826834e-01,...,6.123234e-17,0.961826,-0.273663,0.164595,-0.986361,0.989821,0.142315,0.923880,-3.826834e-01,1.000000
1863260,1863260,4,7,6,3,5,1.0,1.000000,0.642788,7.071068e-01,...,7.071068e-01,0.895163,0.445738,0.475947,-0.879474,0.909632,-0.415415,0.923880,-3.826834e-01,0.923880


In [11]:
df_solve_mi_cf_mmi_select_cp1 = df_solve_mi_cf_mmi_select.copy()
df_solve_mi_cf_mmi_select = df_solve_mi_cf_mmi_select[:1117957].copy()
df_solve_mi_cf_mmi_select['FloodProbability'] = df[:1117957]['FloodProbability']
# df_solve_mi_cf_mmi_select.reset_index(drop=True, inplace=True)
df_solve_mi_cf_mmi_select

,id,MonsoonIntensity,TopographyDrainage,RiverManagement,Deforestation,PoliticalFactors,id_cos,MonsoonIntensity_sin,TopographyDrainage_sin,RiverManagement_sin,...,DeterioratingInfrastructure_sin,DeterioratingInfrastructure_cos,PopulationScore_sin,PopulationScore_cos,WetlandLoss_sin,WetlandLoss_cos,InadequatePlanning_sin,InadequatePlanning_cos,PoliticalFactors_sin,FloodProbability
0,0,5,8,5,8,3,1.000000,0.923880,0.342020,0.923880,...,0.995734,0.092268,0.735724,-0.677282,0.989821,0.142315,0.382683,-9.238795e-01,9.238795e-01,0.445
1,1,6,7,4,4,3,1.000000,0.707107,0.642788,1.000000,...,0.961826,-0.273663,0.837166,0.546948,0.755750,0.654861,1.000000,6.123234e-17,9.238795e-01,0.450
2,2,6,5,6,7,3,1.000000,0.707107,0.984808,0.707107,...,0.798017,-0.602635,0.475947,-0.879474,0.540641,0.841254,0.923880,3.826834e-01,9.238795e-01,0.530
3,3,3,4,6,5,5,1.000000,0.923880,0.984808,0.707107,...,0.995734,0.092268,0.915773,-0.401695,0.989821,0.142315,0.382683,-9.238795e-01,9.238795e-01,0.535
4,4,5,3,2,6,5,1.000000,0.923880,0.866025,0.707107,...,0.995734,0.092268,0.324699,0.945817,0.540641,0.841254,0.923880,3.826834e-01,9.238795e-01,0.415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1117952,1117952,3,3,4,10,4,-0.809026,0.923880,0.866025,1.000000,...,0.673696,0.739009,0.324699,0.945817,0.909632,0.415415,0.707107,-7.071068e-01,1.000000e+00,0.495
1117953,1117953,2,2,4,3,5,-0.809024,0.707107,0.642788,1.000000,...,0.526432,-0.850217,0.969400,0.245485,0.540641,-0.841254,1.000000,6.123234e-17,9.238795e-01,0.480
1117954,1117954,7,3,9,4,4,-0.809022,0.382683,0.866025,-0.382683,...,0.798017,-0.602635,0.996584,-0.082579,0.989821,0.142315,0.707107,7.071068e-01,1.000000e+00,0.485
1117955,1117955,7,3,3,7,4,-0.809020,0.382683,0.866025,0.923880,...,0.995734,0.092268,0.915773,-0.401695,0.909632,-0.415415,0.707107,-7.071068e-01,1.000000e+00,0.495


In [25]:
df

,id,MonsoonIntensity,TopographyDrainage,RiverManagement,Deforestation,Urbanization,ClimateChange,DamsQuality,Siltation,AgriculturalPractices,...,DrainageSystems,CoastalVulnerability,Landslides,Watersheds,DeterioratingInfrastructure,PopulationScore,WetlandLoss,InadequatePlanning,PoliticalFactors,FloodProbability
0,0,5,8,5,8,6,4,4,3,3,...,5,3,3,5,4,7,5,7,3,0.445
1,1,6,7,4,4,8,8,3,5,4,...,7,2,0,3,5,3,3,4,3,0.450
2,2,6,5,6,7,3,7,1,5,4,...,7,3,7,5,6,8,2,3,3,0.530
3,3,3,4,6,5,4,8,4,7,6,...,2,4,7,4,4,6,5,7,5,0.535
4,4,5,3,2,6,4,4,3,3,3,...,2,2,6,6,4,1,2,3,5,0.415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1117952,1117952,3,3,4,10,4,5,5,7,10,...,7,8,7,2,2,1,4,6,4,0.495
1117953,1117953,2,2,4,3,9,5,8,1,3,...,9,4,4,3,7,4,9,4,5,0.480
1117954,1117954,7,3,9,4,6,5,9,1,3,...,5,5,5,5,6,5,5,2,4,0.485
1117955,1117955,7,3,3,7,5,2,3,4,6,...,6,8,5,3,4,6,7,6,4,0.495


# 模型训练

In [26]:
df_test

,id,MonsoonIntensity,TopographyDrainage,RiverManagement,Deforestation,Urbanization,ClimateChange,DamsQuality,Siltation,AgriculturalPractices,...,DrainageSystems,CoastalVulnerability,Landslides,Watersheds,DeterioratingInfrastructure,PopulationScore,WetlandLoss,InadequatePlanning,PoliticalFactors,FloodProbability
0,1117957,4,6,3,5,6,7,8,7,8,...,5,7,5,6,3,6,4,4,5,0.475371
1,1117958,4,4,2,9,5,5,4,7,5,...,4,7,4,5,1,7,4,4,3,0.472348
2,1117959,1,3,6,5,7,2,4,6,4,...,9,2,5,5,2,3,6,8,3,0.505486
3,1117960,2,4,4,6,4,5,4,3,4,...,8,4,6,7,6,4,2,4,4,0.539111
4,1117961,6,3,2,4,6,4,5,5,3,...,3,2,6,4,6,8,4,5,5,0.397402
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
745300,1863257,5,4,8,3,5,4,4,5,5,...,6,1,3,5,6,4,4,6,6,0.606393
745301,1863258,4,4,2,12,4,3,4,3,5,...,7,4,4,3,5,5,3,5,4,0.475305
745302,1863259,5,7,9,5,5,6,7,5,5,...,11,3,11,4,5,9,5,5,4,0.572261
745303,1863260,4,7,6,3,5,2,3,8,6,...,6,8,6,2,3,8,7,5,5,0.476002


In [ ]:
train_x['FloodProbability'] = df['FloodProbability']
train_x

In [ ]:
from autogluon.tabular import TabularPredictor
from autogluon.features.generators import AutoMLPipelineFeatureGenerator
auto_ml_pipeline_feature_generator = AutoMLPipelineFeatureGenerator()
from autogluon.tabular.configs.hyperparameter_configs import get_hyperparameter_config

custom_hyperparameters = get_hyperparameter_config('default')

def get_oof_ts(predictor,test_x):
    predictions_oof = predictor.predict_oof()
    prdict_y = predictor.predict(test_x)
    oof = pd.DataFrame({'oofs': predictions_oof, 'predict_y': prdict_y}).to_csv('model{}.csv',index=False)
    return predictions_oof,prdict_y
# 训练集原始为 117957 条数据
# 初始训练
# df_train = df # df_solve_mi_cf_mmi_select[:1117957]
# df_test2 =  df_test  # df_solve_mi_cf_mmi_select  #[:2000]


# predictor = TabularPredictor(label="FloodProbability").fit(
#     train_x,#df_train,
#     presets= 'best_quality', # ['optimize_for_deployment'],  #'best_quality',
# #     time_limit=60,
#     # num_stack_levels=1,
#     # num_bag_folds=3,
# #     time_limit=3600*15
# )
# predictions = predictor.predict(df_test2)

predictor = TabularPredictor(
    label="FloodProbability",
    problem_type='regression',
    eval_metric='r2'
).fit(
    train_x, #df, #df_train,
    presets='best_quality',
#     unlabeled_data = df_test2,   #df_test2.drop(columns='Survived'), #df_train[800:].drop(columns='Survived'),
    # auto_stack=True,
    feature_generator=auto_ml_pipeline_feature_generator,
    ag_args_ensemble = {'fold_fitting_strategy':'sequential_local'},
    # dynamic_stacking=True,
#     num_stack_levels=1,
    num_bag_folds=10,
    # time_limit=60*4,
#     included_model_types = ['XGB','KNN',''],
#     excluded_model_types = ['NN_TORCH'],
    # use_bag_holdout=True,
    # hyperparameters=custom_hyperparameters,
    # hyperparameter_tune_kwargs='auto',#hyperparameter_tune_kwargs,
    keep_only_best = True,
)
aa,predictions = get_oof_ts(predictor,test_x)

In [ ]:
predictions

In [ ]:
predictor.leaderboard()

In [14]:
# from sklearn.model_selection import cross_val_score
# cross_val_score(predictor, df_train.drop('FloodProbability',axis=1), df_train['FloodProbability'], scoring = 'accuracy', cv = 10).mean()

In [15]:
# predictor.evaluate(df_test2)


KeyboardInterrupt



In [16]:
df_test

,id,MonsoonIntensity,TopographyDrainage,RiverManagement,Deforestation,Urbanization,ClimateChange,DamsQuality,Siltation,AgriculturalPractices,...,IneffectiveDisasterPreparedness,DrainageSystems,CoastalVulnerability,Landslides,Watersheds,DeterioratingInfrastructure,PopulationScore,WetlandLoss,InadequatePlanning,PoliticalFactors
0,1117957,4,6,3,5,6,7,8,7,8,...,8,5,7,5,6,3,6,4,4,5
1,1117958,4,4,2,9,5,5,4,7,5,...,2,4,7,4,5,1,7,4,4,3
2,1117959,1,3,6,5,7,2,4,6,4,...,7,9,2,5,5,2,3,6,8,3
3,1117960,2,4,4,6,4,5,4,3,4,...,7,8,4,6,7,6,4,2,4,4
4,1117961,6,3,2,4,6,4,5,5,3,...,4,3,2,6,4,6,8,4,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
745300,1863257,5,4,8,3,5,4,4,5,5,...,5,6,1,3,5,6,4,4,6,6
745301,1863258,4,4,2,12,4,3,4,3,5,...,3,7,4,4,3,5,5,3,5,4
745302,1863259,5,7,9,5,5,6,7,5,5,...,6,11,3,11,4,5,9,5,5,4
745303,1863260,4,7,6,3,5,2,3,8,6,...,6,6,8,6,2,3,8,7,5,5


In [17]:
df_train['FloodProbability']

0          0.445
1          0.450
2          0.530
3          0.535
4          0.415
           ...  
1117952    0.495
1117953    0.480
1117954    0.485
1117955    0.495
1117956    0.560
Name: FloodProbability, Length: 1117957, dtype: float64

In [18]:
pl = predictions
# pl = list(int(i) for i in predictions)
pl

0          0.475371
1          0.472348
2          0.505486
3          0.539111
4          0.397402
             ...   
1117952    0.498129
1117953    0.485137
1117954    0.495914
1117955    0.510269
1117956    0.570967
Name: FloodProbability, Length: 1117957, dtype: float32

In [19]:
df_test['FloodProbability'] = pl
df_test
df_sub = df_test[['id','FloodProbability']]
df_sub

,id,FloodProbability
0,1117957,0.475371
1,1117958,0.472348
2,1117959,0.505486
3,1117960,0.539111
4,1117961,0.397402
...,...,...
745300,1863257,0.606393
745301,1863258,0.475305
745302,1863259,0.572261
745303,1863260,0.476002


In [20]:
df_sub.to_csv('submission.csv',index=False)

In [21]:
cc = df_sub['FloodProbability'].value_counts()
cc

FloodProbability
0.550441    7
0.522796    7
0.522385    6
0.522915    6
0.485179    6
           ..
0.471643    1
0.472415    1
0.562462    1
0.660151    1
0.476002    1
Name: count, Length: 656643, dtype: int64

In [22]:
!kaggle competitions submit -c playground-series-s4e5 -f submission.csv -m "Message"

Successfully submitted to Regression with a Flood Prediction Dataset



  0%|          | 0.00/13.9M [00:00<?, ?B/s]
  0%|          | 8.00k/13.9M [00:00<11:48, 20.5kB/s]
 32%|███▏      | 4.50M/13.9M [00:00<00:01, 7.44MB/s]
 53%|█████▎    | 7.30M/13.9M [00:00<00:00, 9.80MB/s]
 75%|███████▍  | 10.4M/13.9M [00:01<00:00, 12.0MB/s]
 90%|████████▉ | 12.4M/13.9M [00:01<00:00, 11.9MB/s]
100%|██████████| 13.9M/13.9M [00:04<00:00, 3.40MB/s]
